In [ ]:
from jqdata import *
import statsmodels.api as sm
import pandas as pd
import numpy as np

# 初始化函数，设定基准等等
def initialize(context):
    # 设定沪深300作为基准
    set_benchmark('601398.XSHG')
    # 开启动态复权模式(真实价格)
    set_option('use_real_price', True)
    # 输出内容到日志 log.info()
    log.info('初始函数开始运行且全局只运行一次')
    # 过滤掉order系列API产生的比error级别低的log
    log.set_level('order', 'error')
    # 开启防未来函数
    set_option("avoid_future_data", True)

    ### 股票相关设定 ###
    # 股票类每笔交易时的手续费是：买入时佣金万分之三，卖出时佣金万分之三加千分之一印花税, 每笔交易佣金最低扣5块钱
    set_order_cost(OrderCost(close_tax=0.001, open_commission=0.0003, close_commission=0.0003, min_commission=5), type='stock')
    ## 运行函数（reference_security为运行时间的参考标的；传入的标的只做种类区分，因此传入'000300.XSHG'或'510300.XSHG'是一样的）
    g.stocks=['601398.XSHG', '601939.XSHG']
    # y=a*x+b
    # 储存方程组系数
    g.a=0
    g.b=0
    g.std=0
      # 开盘时运行
    run_daily(trade, time='9:30', reference_security='000300.XSHG')
    run_daily(trade2, time='9:35', reference_security='000300.XSHG')
    # # 盘中时运行
    # run_daily(func, time='every_bar', reference_security='000300.XSHG')
      # 收盘后运行
    run_daily(before_market_open, time='before_open', reference_security='000300.XSHG')

## 开盘前运行函数
def before_market_open(context):
    # 输出运行时间
    # log.info('函数运行时间(before_market_open)：'+str(context.current_dt.time()))
    dt=context.previous_date.strftime("%Y-%m-%d")
    code = '601398.XSHG'
    price_df = get_price(code, end_date=dt, frequency='daily', 
                      fields=[ 'open', 'high', 'low', 'close'], skip_paused=False, fq='pre', count=1000)
    code2 = '601939.XSHG'
    price_df2 = get_price(code2, end_date=dt, frequency='daily', 
                      fields=[ 'open', 'high', 'low', 'close'], skip_paused=False, fq='pre', count=1000)                  
    df=price_df.merge(price_df2,left_index=True, right_index=True)
    df.dropna(axis=0, how='any', inplace=True)
    x=df.close_x
    y=df.close_y
    #一元方程的非协整检验
    coint_result = sm.tsa.stattools.coint(x, y)
    X = sm.add_constant(x) #增加常数项
    #进行线性回归拟合
    result = (sm.OLS(y,X)).fit()
    g.a=result.params[1]
    g.b=result.params[0]
    # 计算预测值
    df['y1']=result.fittedvalues
    # 计算残差项
    df['residual']=df.close_y-df.y1
    # 输出残差标准差
    g.std=np.std(df.residual)
    g.ord1=False
    g.ord2=False
   
    # 给微信发送消息（添加模拟交易，并绑定微信生效）
    # send_message('美好的一天~')
def trade(context):
    close_data = get_bars(g.stocks, count=1, unit='1d', fields=['close'])
    if context.portfolio.positions_value==0:
        # 取得当前的现金
        cash = context.portfolio.available_cash
        if g.a*close_data[g.stocks[0]]['close'][0]+g.b>=close_data[g.stocks[1]]['close'][0]:
            order_value(g.stocks[0], cash)
            g.ord1=True
        else:
            order_value(g.stocks[1], cash)
            g.ord2=True
    else:
        if g.a*close_data[g.stocks[0]]['close'][0]+g.b+1*g.std<close_data[g.stocks[1]]['close'][0]:
            order_target(g.stocks[1], 0)
            cash = context.portfolio.available_cash
            order_value(g.stocks[0], cash)
            g.ord1=True
        elif g.a*close_data[g.stocks[0]]['close'][0]+g.b-1*g.std>close_data[g.stocks[1]]['close'][0]:
            order_target(g.stocks[0], 0)
            cash = context.portfolio.available_cash
            order_value(g.stocks[1], cash)
            g.ord2=True

def trade2(context):
    cash = context.portfolio.available_cash
    if g.ord1:
        order_value(g.stocks[0], cash)
        g.ord1=False
    elif g.ord2:
        order_value(g.stocks[1], cash)
        g.ord2=False
        
    
## 开盘时运行函数
def market_open(context):
    log.info('函数运行时间(market_open):'+str(context.current_dt.time()))
    security = g.security
    # 获取股票的收盘价
    close_data = get_bars(security, count=5, unit='1d', fields=['close'])
    # 取得过去五天的平均价格
    MA5 = close_data['close'].mean()
    # 取得上一时间点价格
    current_price = close_data['close'][-1]
    # 取得当前的现金
    cash = context.portfolio.available_cash

    # 如果上一时间点价格高出五天平均价1%, 则全仓买入
    if (current_price > 1.01*MA5) and (cash > 0):
        # 记录这次买入
        log.info("价格高于均价 1%%, 买入 %s" % (security))
        # 用所有 cash 买入股票
        order_value(security, cash)
    # 如果上一时间点价格低于五天平均价, 则空仓卖出
    elif current_price < MA5 and context.portfolio.positions[security].closeable_amount > 0:
        # 记录这次卖出
        log.info("价格低于均价, 卖出 %s" % (security))
        # 卖出所有股票,使这只股票的最终持有量为0
        order_target(security, 0)

## 收盘后运行函数
def after_market_close(context):
    log.info(str('函数运行时间(after_market_close):'+str(context.current_dt.time())))
    #得到当天所有成交记录
    trades = get_trades()
    for _trade in trades.values():
        log.info('成交记录：'+str(_trade))
    log.info('一天结束')
    log.info('##############################################################')
